In [7]:
# System Info
import sys       # Access system-specific parameters and Python version
import os        # Interface with the operating system
import platform  # Get underlying platform and hardware info

# Data Processing
import numpy  # Numerical operations and arrays
import pandas as pd  # Data manipulation and analysis using DataFrames

# Visualization
import matplotlib  # Basic plotting library
import seaborn as sns  # Statistical data visualization on top of matplotlib

# Machine Learning & NLP
import sklearn  # Machine learning tools and TF-IDF baseline
import transformers  # Huggingface library for pretrained transformer models like BERT
import datasets  # Huggingface datasets library for loading NLP datasets

# Deep Learning
import torch  # PyTorch library for tensor operations and model training

# Progress Bars
import tqdm  # Show progress bars for loops and training

# Display versions
print(f"Python version      : {sys.version}")
print(f"NumPy version       : {numpy.__version__}")
print(f"Pandas version      : {pd.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"Matplotlib version  : {matplotlib.__version__}")
print(f"Seaborn version     : {sns.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"Huggingface Datasets: {datasets.__version__}")
print(f"PyTorch version     : {torch.__version__}")
print(f"TQDM version        : {tqdm.__version__}")
print(f"CPU architecture    : {platform.processor() or platform.machine()}")

Python version      : 3.10.18 (main, Jun  5 2025, 08:37:47) [Clang 14.0.6 ]
NumPy version       : 2.2.6
Pandas version      : 2.3.1
Scikit-learn version: 1.7.1
Matplotlib version  : 3.10.5
Seaborn version     : 0.13.2
Transformers version: 4.54.1
Huggingface Datasets: 4.0.0
PyTorch version     : 2.7.1
TQDM version        : 4.67.1
CPU architecture    : arm
